In [71]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.utils.extmath import safe_sparse_dot

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/22n0457/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
imdb_sup_df=pd.read_csv("/home/22n0457/imdb_sup.csv")

In [73]:
imdb_sup_df.head(10)

,Review,Rating,Sentiment
0,"Kurt Russell's chameleon-like performance, cou...",10,1
1,It was extremely low budget(it some scenes it ...,8,1
2,James Cagney is best known for his tough chara...,8,1
3,"Following the brilliant ""Goyôkiba"" (aka. ""Hanz...",8,1
4,One of the last classics of the French New Wav...,10,1
5,Having just watched this film again from a 199...,10,1
6,The Straight Story is a truly beautiful movie ...,10,1
7,Four teenage girlfriends drive to Fort Laurdal...,7,1
8,"I haven't seen all of Jess Franco's movies, I ...",9,1
9,What's in a name? If the name is Jerry Bruckhe...,7,1


In [74]:
imdb_sup_df=imdb_sup_df.iloc[:1000,:]

In [75]:
def text_clean(message):
  '''
  message = "#'This', is $string #with punction, <br /> @'html_tag' and actual message also!"

  return 'string punction html_tag actual message also'
  '''

  html_tag = '<br />'
  message = message.replace(html_tag,'')  # remove html tag
  message = re.sub(r'[^\w\s]', '', message)   # remove punctiation
  message = message.lower()
  message = [word for word in message.split() if word not in stopwords.words('english')]
  message = ' '.join(message)

  return message

In [76]:
test_str = "#'This', is $string #with punction, <br /> @'html_tag' and actual message also!"
text_clean(test_str)

'string punction html_tag actual message also'

In [77]:
imdb_sup_df['Review'] = imdb_sup_df['Review'].apply(text_clean)

In [78]:
imdb_sup_df

,Review,Rating,Sentiment
0,kurt russells chameleonlike performance couple...,10,1
1,extremely low budgetit scenes looks like recor...,8,1
2,james cagney best known tough characters gangs...,8,1
3,following brilliant goyôkiba aka hanzo razor s...,8,1
4,one last classics french new wave direction ci...,10,1
...,...,...,...
995,watching midnight cowboy like taking mastercla...,10,1
996,ladies man suffers common problem among movies...,7,1
997,picked movie cover even knowing watched laughe...,10,1
998,movie awesome nonstop laugh riot incorporating...,7,1


In [81]:
from nltk.stem.porter import PorterStemmer

porterstemmer = PorterStemmer()

def steming(message):
  return[porterstemmer.stem(word) for word in message.split()]

In [82]:
# 1-2 minute
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=steming) # here stemming is our user defined function as above 

x = tfidf.fit_transform(imdb_sup_df['Review']).toarray()
y = imdb_sup_df['Rating'].values

In [83]:
y

array([10,  8,  8,  8, 10, 10, 10,  7,  9,  7,  7, 10, 10, 10,  7,  8, 10,
       10,  8,  7,  8, 10,  9,  8,  8,  9,  9,  8, 10, 10,  9,  7,  7, 10,
        8,  9,  9,  8,  7, 10,  8,  8,  9,  9, 10, 10,  9,  8, 10, 10,  7,
        9, 10, 10,  8,  8, 10,  8,  8, 10,  9, 10,  9, 10,  7,  7, 10,  7,
        9,  7,  7, 10, 10,  8,  8, 10,  8, 10, 10,  8,  8,  8,  9,  8,  8,
        7, 10,  8,  7,  8,  7, 10, 10, 10,  9,  8, 10,  7,  9,  7,  7,  7,
        8, 10,  8,  8,  7,  7, 10,  9,  7, 10,  8, 10,  9,  8, 10, 10,  8,
        7,  8,  8,  9,  7, 10,  8, 10, 10,  7, 10,  8,  9,  9, 10,  9, 10,
       10,  9,  7, 10,  9,  9,  7, 10, 10,  7,  9, 10,  8,  8,  7,  7,  8,
        8,  8, 10,  8,  7,  7,  7, 10,  9,  8, 10,  7, 10,  7,  7, 10,  9,
        8,  7,  9,  7, 10,  8, 10, 10, 10,  7,  8,  7,  8,  9,  7,  8,  9,
       10,  9, 10,  9,  9,  9,  7, 10,  7,  8, 10, 10,  7, 10,  8,  8, 10,
       10,  7,  8,  7,  7, 10,  8,  9, 10,  7, 10,  8,  8,  9,  7,  8,  9,
       10, 10,  7, 10, 10

In [84]:
def widrowhoff(train,y,t,c,m):
    w=np.array([0 for i in range(np.shape(train)[1])])
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for s in range(m):
        for i in range(1,t+1):
            x=np.array(train[i-1])
            y_bart=safe_sparse_dot(w,x)
            #if i%1000==0:
            if i%2==0:
                y_bar=[]
                for t in range(1,i+1):
                    x=np.array(train[t-1])
                    y_bart=safe_sparse_dot(w,x)
                    y_bar.append(y_bart)
                l=sum([np.absolute(y[j-1]-y_bar[j-1]) for j in range(1,i+1)])/i
                print(f"loss in round {i} is ",l)
                loss.append(l)
            w=np.subtract(w,c*(safe_sparse_dot(w,x)-y[i-1])*x)
    return loss,w

In [85]:
def pranking(train,y,t,m):
    w=np.zeros(train.shape[1])
    b = np.zeros(len(np.unique(y)) - 1)
    b = np.append(b, np.inf)
    loss=[]
    label=range(1,len(np.unique(y))+1)
    unique_y = np.sort(np.unique(y))
    y = np.array([np.where(unique_y == labels)[0][0] + 1 for labels in y]) 
    def predict(x, w, b):
        scores = safe_sparse_dot(x, w) < b
        return np.argmax(scores) + 1
    for s in range(m):
        for i in range(1,t+1):
            x=train[i-1]
            y_bar=predict(x, w, b)
            if y_bar !=y[i-1]:
                y_rt=np.where(y[i-1]<=label[:-1],-1,1)
                t_rt=np.where((safe_sparse_dot(x,w)-b[:-1])*y_rt <=0,y_rt,0)
                w=w+sum(t_rt)*x
                b[:-1] -= t_rt
            #predicting score
            #if i%1000==0:
            if i%2==0:
                y_bar1=np.array([predict(train[j-1],w,b) for j in range(1,i+1)])
                l = np.mean(np.abs(y[:i] - y_bar1)) #loss function
                print(f"loss in round {i} is ",l)
                loss.append(l)
    return loss,w,b

In [86]:
def uniformmulticlassalgo(feature,y,maxiter,d):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    loss=[]
    def taugenerate(h,e,yt):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        for k in e:
            t[k-1]=-(1/len(e))
        return t
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for s in range(d):
        for i in range(1,maxiter+1):
            x=feature[i-1]
            ybart=np.argmax([safe_sparse_dot(m[j],x) for j in range(len(label))])+1
            yt=y[i-1]
            e=[]
            for r in range(1,len(label)+1):
                if  r!=yt:
                    if safe_sparse_dot(m[int(r-1)],x)>=safe_sparse_dot(m[int(yt-1)],x):
                        e.append(r)
            if len(e)!=0:
                tau=taugenerate(len(label),e,yt)
                for k in label:
                    m[k-1]=np.add(m[k-1],tau[k-1]*x)
            #if i%1000==0:
            if i%2==0:
                y_bar1=[]       
                for t in range(1,i+1):
                    x=feature[t-1]
                    ybart=np.argmax([safe_sparse_dot(m[j],x) for j in range(len(label))])+1
                    y_bar1.append(ybart)
                l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
                print(f"loss in round {i} is ",l)
                loss.append(l)
    return loss,m         

In [87]:
def worstmulticlassalgo(feature,y,maxiter,d):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt,label):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        k=np.argsort([safe_sparse_dot(m[j],x) for j in range(h)])[-2]
        t[k]=-1
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for s in range(d):
        for i in range(1,maxiter+1):
            x=feature[i-1]
            ybart=np.argmax([safe_sparse_dot(m[j],x) for j in range(len(label))])+1
            yt=y[i-1]
            e=[]
            for r in range(1,len(label)+1):
                if  r!=yt:
                    if safe_sparse_dot(m[int(r-1)],x)>=safe_sparse_dot(m[int(yt-1)],x):
                        e.append(r)
            if len(e)!=0:
                tau=taugenerate(len(label),m,x,yt,label)
                for k in label:
                    m[k-1]=np.add(m[k-1],tau[k-1]*x)
            #if i%1000==0:
            if i%2==0:
                y_bar1=[]       
                for t in range(1,i+1):
                    x=feature[t-1]
                    ybart=np.argmax([safe_sparse_dot(m[j],x) for j in range(len(label))])+1
                    y_bar1.append(ybart)
                l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
                print(f"loss in round {i} is ",l)
                loss.append(l)
    return loss,m

In [88]:
def vimulticlassalgo(feature,y,maxiter,d):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    loss=[]
    y_bar1=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        for i in range(h):
            if i!=int(yt-1):
                if safe_sparse_dot(m[i],x)-safe_sparse_dot(m[int(yt-1)],x)>0:
                    num=safe_sparse_dot(m[i],x)-safe_sparse_dot(m[int(yt-1)],x)
                else:
                    num=0
                den=0
                for k in range(h):
                    if safe_sparse_dot(m[k],x)-safe_sparse_dot(m[int(yt-1)],x)>0:
                        den+=safe_sparse_dot(m[k],x)-safe_sparse_dot(m[int(yt-1)],x)
                if den==0:
                    den=1
                t[i]=-(num/den)     
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for s in range(d):
        for i in range(1,maxiter+1):
            x=feature[i-1]
            ybart=np.argmax([safe_sparse_dot(m[j],x) for j in range(len(label))])+1
            yt=y[i-1]
            e=[]
            for r in range(1,len(label)+1):
                if  r!=yt:
                    if safe_sparse_dot(m[r-1],x)>=safe_sparse_dot(m[int(yt-1)],x):
                        e.append(r)
            if len(e)!=0:
                tau=taugenerate(len(label),m,x,yt)
                for k in label:
                    m[k-1]=np.add(m[k-1],tau[k-1]*x)
            #if i%1000==0:
            if i%2==0:
                y_bar1=[]       
                for t in range(1,i+1):
                    x=feature[t-1]
                    ybart=np.argmax([safe_sparse_dot(m[j],x) for j in range(len(label))])+1
                    y_bar1.append(ybart)
                l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
                print(f"loss in round {i} is ",l)
                loss.append(l)
    return loss,m

In [89]:
def mira(feature,y,maxiter,d):
    m=np.ones((len(np.unique(y)),len(feature[0])))
    y_bar1=[]
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt,ybart):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        tau=np.minimum(0.001, (((m[int(ybart-1)]-m[int(yt-1)])*x)+1.0)/(2.0*(safe_sparse_dot(x,x))))
        t[int(yt-1)]=tau
        t[int(ybart-1)]=-tau
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for s in range(d):
        for i in range(1,maxiter+1):
            x=feature[i-1]
            ybart=np.argmax([safe_sparse_dot(m[j],x) for j in range(len(label))])+1
            yt=y[i-1]
            tau=taugenerate(len(label),m,x,yt,ybart)
            for k in label:
                m[k-1]=np.add(m[k-1],tau[k-1]*x)
            #if i%1000==0:
            if i%2==0:
                y_bar1=[]       
                for t in range(1,i+1):
                    x=feature[t-1]
                    ybart=np.argmax([safe_sparse_dot(m[j],x) for j in range(len(label))])+1
                    y_bar1.append(ybart)
                l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
                print(f"loss in round {i} is ",l)
                loss.append(l)
    return loss,m

In [90]:
loss1,w1=widrowhoff(x,y,len(y),0.001,20)

loss in round 2 is  2.997972442541511
loss in round 4 is  2.497834941111342
loss in round 6 is  2.997177844755044
loss in round 8 is  2.871500609992203
loss in round 10 is  2.6964861221163647
loss in round 12 is  2.6633010270263573
loss in round 14 is  2.853306014805497
loss in round 16 is  2.6835044122142215
loss in round 18 is  2.829007408025245
loss in round 20 is  2.6954612117129524
loss in round 22 is  2.722780564758887
loss in round 24 is  2.7034739473168283
loss in round 26 is  2.6874196708415874
loss in round 28 is  2.673459912172404
loss in round 30 is  2.7614197027806884
loss in round 32 is  2.713290797693657
loss in round 34 is  2.7004470919275083
loss in round 36 is  2.688735197523121
loss in round 38 is  2.678232293800933
loss in round 40 is  2.66895884875066
loss in round 42 is  2.6366750353915114
loss in round 44 is  2.6526360030948117
loss in round 46 is  2.7106641124073483
loss in round 48 is  2.7013331184834564
loss in round 50 is  2.752819343868059
loss in round 52 i

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/imdb/loss1.csv',loss1, delimiter=',')
np.savetxt('/home/22n0457/imdb/w1.csv',w1, delimiter=',')

In [91]:
loss2,w2,b2=pranking(x,y,len(y),20)

loss in round 2 is  1.0
loss in round 4 is  0.5
loss in round 6 is  0.6666666666666666
loss in round 8 is  0.625
loss in round 10 is  1.1
loss in round 12 is  0.8333333333333334
loss in round 14 is  0.7142857142857143
loss in round 16 is  0.875
loss in round 18 is  0.5555555555555556
loss in round 20 is  0.8
loss in round 22 is  0.6363636363636364
loss in round 24 is  0.7083333333333334
loss in round 26 is  0.6538461538461539
loss in round 28 is  0.6071428571428571
loss in round 30 is  0.6666666666666666
loss in round 32 is  0.71875
loss in round 34 is  0.5294117647058824
loss in round 36 is  0.6666666666666666
loss in round 38 is  0.6052631578947368
loss in round 40 is  0.675
loss in round 42 is  0.6666666666666666
loss in round 44 is  0.6363636363636364
loss in round 46 is  0.717391304347826
loss in round 48 is  0.5625
loss in round 50 is  0.58
loss in round 52 is  0.5384615384615384
loss in round 54 is  0.8148148148148148
loss in round 56 is  0.625
loss in round 58 is  0.63793103448

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/imdb/loss2.csv',loss2, delimiter=',')
np.savetxt('/home/22n0457/imdb/w2.csv',w2, delimiter=',')
np.savetxt('/home/22n0457/imdb/b2.csv',b2, delimiter=',')

In [92]:
loss3,m1=uniformmulticlassalgo(x,y,len(y),20)

loss in round 2 is  0.0
loss in round 4 is  0.0
loss in round 6 is  0.0
loss in round 8 is  0.5
loss in round 10 is  0.5
loss in round 12 is  0.5833333333333333
loss in round 14 is  0.5
loss in round 16 is  0.25
loss in round 18 is  0.2222222222222222
loss in round 20 is  0.2
loss in round 22 is  0.18181818181818182
loss in round 24 is  0.20833333333333331
loss in round 26 is  0.23076923076923078
loss in round 28 is  0.3214285714285714
loss in round 30 is  0.2
loss in round 32 is  0.25
loss in round 34 is  0.14705882352941177
loss in round 36 is  0.2777777777777778
loss in round 38 is  0.15789473684210525
loss in round 40 is  0.15000000000000002
loss in round 42 is  0.14285714285714285
loss in round 44 is  0.29545454545454547
loss in round 46 is  0.10869565217391304
loss in round 48 is  0.25
loss in round 50 is  0.2
loss in round 52 is  0.25
loss in round 54 is  0.25925925925925924
loss in round 56 is  0.23214285714285712
loss in round 58 is  0.20689655172413793
loss in round 60 is  0.

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/imdb/loss3.csv',loss3, delimiter=',')

In [93]:
loss4,m2=worstmulticlassalgo(x,y,len(y),20)

loss in round 2 is  1.0
loss in round 4 is  0.5
loss in round 6 is  0.8333333333333333
loss in round 8 is  0.875
loss in round 10 is  0.7000000000000001
loss in round 12 is  0.8333333333333333
loss in round 14 is  0.7142857142857142
loss in round 16 is  0.6875
loss in round 18 is  0.611111111111111
loss in round 20 is  0.55
loss in round 22 is  0.5
loss in round 24 is  0.875
loss in round 26 is  0.7307692307692308
loss in round 28 is  0.6785714285714285
loss in round 30 is  0.6666666666666666
loss in round 32 is  0.625
loss in round 34 is  0.5882352941176471
loss in round 36 is  0.611111111111111
loss in round 38 is  0.5
loss in round 40 is  0.6000000000000001
loss in round 42 is  0.5238095238095237
loss in round 44 is  0.5
loss in round 46 is  0.5217391304347826
loss in round 48 is  0.5
loss in round 50 is  0.54
loss in round 52 is  0.5
loss in round 54 is  0.48148148148148145
loss in round 56 is  0.46428571428571425
loss in round 58 is  0.48275862068965514
loss in round 60 is  0.5333

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/imdb/loss4.csv',loss4, delimiter=',')

In [67]:
loss5,m3=vimulticlassalgo(x,y,len(y),20)

loss in round 2 is  0.0
loss in round 4 is  0.0
loss in round 6 is  0.6666666666666666
loss in round 8 is  0.5
loss in round 10 is  0.5
loss in round 12 is  0.41666666666666663
loss in round 14 is  0.5
loss in round 16 is  0.625
loss in round 18 is  0.38888888888888884
loss in round 20 is  0.25
loss in round 22 is  0.13636363636363635
loss in round 24 is  0.125
loss in round 26 is  0.15384615384615385
loss in round 28 is  0.2857142857142857
loss in round 30 is  0.2
loss in round 32 is  0.25
loss in round 34 is  0.23529411764705882
loss in round 36 is  0.2777777777777778
loss in round 38 is  0.2631578947368421
loss in round 40 is  0.25
loss in round 42 is  0.21428571428571427
loss in round 44 is  0.2727272727272727
loss in round 46 is  0.19565217391304346
loss in round 48 is  0.2708333333333333
loss in round 50 is  0.28
loss in round 52 is  0.21153846153846156
loss in round 54 is  0.2222222222222222
loss in round 56 is  0.19642857142857142
loss in round 58 is  0.27586206896551724
loss i

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/imdb/loss5.csv',loss5, delimiter=',')

In [96]:
loss6,m4=mira(x,y,len(y),20)

loss in round 2 is  0.0
loss in round 4 is  0.5
loss in round 6 is  0.8333333333333333
loss in round 8 is  0.625
loss in round 10 is  0.4
loss in round 12 is  0.3333333333333333
loss in round 14 is  0.5
loss in round 16 is  0.25
loss in round 18 is  0.2777777777777778
loss in round 20 is  0.30000000000000004
loss in round 22 is  0.4090909090909091
loss in round 24 is  0.3333333333333333
loss in round 26 is  0.2692307692307693
loss in round 28 is  0.3571428571428571
loss in round 30 is  0.43333333333333335
loss in round 32 is  0.375
loss in round 34 is  0.4411764705882353
loss in round 36 is  0.2777777777777778
loss in round 38 is  0.42105263157894735
loss in round 40 is  0.375
loss in round 42 is  0.42857142857142855
loss in round 44 is  0.29545454545454547
loss in round 46 is  0.45652173913043476
loss in round 48 is  0.2708333333333333
loss in round 50 is  0.46
loss in round 52 is  0.46153846153846156
loss in round 54 is  0.38888888888888884
loss in round 56 is  0.4821428571428571
los

KeyboardInterrupt: 

In [95]:
np.savetxt('/home/22n0457/imdb/loss6.csv',loss6, delimiter=',')

NameError: name 'loss6' is not defined

In [97]:
def pranking_updated(train,y,t,m):
    w=np.zeros(train.shape[1])
    b = np.zeros(len(np.unique(y)) - 1)
    b = np.append(b, np.inf)
    loss=[]
    label=range(1,len(np.unique(y))+1)
    unique_y = np.sort(np.unique(y))
    y = np.array([np.where(unique_y == labels)[0][0] + 1 for labels in y]) 
    def predict(x, w, b):
        scores = np.absolute(safe_sparse_dot(x, w)-b)
        return np.argmin(scores) + 1
    for s in range(m):
        for i in range(1,t+1):
            x=train[i-1]
            y_bar=predict(x, w, b)
            if y_bar !=y[i-1]:
                y_rt=np.where(y[i-1]<=label[:-1],-1,1)
                t_rt=np.where((safe_sparse_dot(x,w)-b[:-1])*y_rt <=0,y_rt,0)
                w=w+sum(t_rt)*x
                b[:-1] -= t_rt
            #predicting score
            if i%1000==0:
                y_bar1=np.array([predict(train[j-1],w,b) for j in range(1,i+1)])
                l = np.mean(np.abs(y[:i] - y_bar1)) #loss function
                print(f"loss in round {i} is ",l)
                loss.append(l)
    return loss,w,b

In [98]:
loss7,w21,b21=pranking_updated(x,y,len(y),20)

loss in round 1000 is  1.659
loss in round 1000 is  1.078
loss in round 1000 is  1.05
loss in round 1000 is  0.55
loss in round 1000 is  0.533
loss in round 1000 is  0.51
loss in round 1000 is  0.467
loss in round 1000 is  0.486
loss in round 1000 is  0.449
loss in round 1000 is  0.433
loss in round 1000 is  0.568
loss in round 1000 is  0.537
loss in round 1000 is  0.535
loss in round 1000 is  0.534
loss in round 1000 is  0.54
loss in round 1000 is  0.4
loss in round 1000 is  0.398
loss in round 1000 is  0.533
loss in round 1000 is  0.533
loss in round 1000 is  0.533


In [ ]:
np.savetxt('/home/22n0457/imdb/loss7.csv',loss7, delimiter=',')

In [99]:
from sklearn.utils.extmath import safe_sparse_dot

In [38]:
class OrdinalRegressionNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * 0.01
        self.b2 = np.zeros((1, self.output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, X):
        self.Z1 = safe_sparse_dot(X, self.W1) + self.b1
        self.A1 = self.sigmoid(self.Z1)
        self.Z2 = safe_sparse_dot(self.A1, self.W2) + self.b2
        self.O = [self.sigmoid(z) for z in self.Z2]
        return self.O

    def backward(self, X, y_true, y_pred, learning_rate):
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        # calculate errors and gradients
        delta2 = y_pred - y_true
        dW2 = safe_sparse_dot(self.A1.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)
        delta1 = safe_sparse_dot(delta2, self.W2.T) * (self.A1 * (1 - self.A1))
        dW1 = safe_sparse_dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0)

        # update weights and biases
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1

    def train(self, X, y, learning_rate=0.01, epochs=100):
        n_samples = X.shape[0]
        loss=[]

        # encode ordinal categories in target vectors
        y_encoded = np.zeros((n_samples, self.output_size))
        for i, y_i in enumerate(y):
            y_encoded[i, :int(y_i)] = 1

        # train the network
        for epoch in range(epochs):
            y_pred = self.forward(X)
            self.backward(X, y_encoded, y_pred, learning_rate)
            y_pred = [np.argmax(o) + 1 for o in y_pred]
            loss.append(np.sum(np.absolute(y_pred-y))/len(y))
        return loss

    def predict(self, X):
        y_pred = self.forward(X)
        y_pred = [np.argmax(o) + 1 for o in y_pred]
        return y_pred

In [100]:
label=range(1,len(np.unique(y))+1)
unique_y = np.sort(np.unique(y))
y = np.array([np.where(unique_y == labels)[0][0] + 1 for labels in y])

In [101]:
model = OrdinalRegressionNetwork(input_size=np.shape(x)[1], hidden_size=2**7, output_size=int(len(np.unique(y))))
#loss=model.train(x, y, learning_rate=0.1, epochs=1000)
loss=model.train(x[:100], y[:100], learning_rate=0.1, epochs=1)

In [ ]:
np.savetxt('/home/22n0457/imdb/loss8.csv',loss, delimiter=',')

In [102]:
loss

[1.32]

In [ ]:
5